# Analyzing nanopore sequencing data for AMBIC

## Set up
Necessary softwares should be installed already in here.  
They are described in README.md as well as   
I made a snakemake pipeline for ease of use - I will go through each step using commands as well as snakemake.

In [9]:
# first set working directory
workdir=~/Data_tmp
codedir=$PWD
reference=/mnt/ref/4_0cdhfr_vrc01wtg1m3_dgv.fa
reference=/mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.fa
#reference=/mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.fa
threads=12
sample=CHOK1IgG

## Alignment
There are two main aligners we use for nanopore analysis : minimap2 and ngmlr
### minimap2
First make index

In [10]:
mmi=${reference%.fa}.map_ont.mmi
if [ ! -e $mmi ]; then
    minimap2 -x map-ont -d $mmi $reference
fi

In [12]:
ls -lh $mmi

-rw-r--r-- 1 jupyter-isac jupyter-isac 5.6G Oct 23 11:59 /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.map_ont.mmi


Then align

In [8]:
[ -e bam ]||mkdir bam
fq=$workdir/reads/$sample.fastq.gz
outpre=$workdir/bam/$sample.minimap2
output=$outpre.sorted.bam
minimap2 --MD -L -t $threads -a $mmi ${fq} |\
    samtools view -q 20 -b - |\
    samtools sort -T $outpre.sorting -o ${output} &&\
    samtools index ${output}

[M::main::6.110*1.00] loaded/built the index for 1831 target sequence(s)
[M::mm_mapopt_update::7.736*1.00] mid_occ = 490
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1831
[M::mm_idx_stat::8.605*1.00] distinct minimizers: 92926228 (39.14% are singletons); average occurrences: 4.825; average spacing: 5.284
[M::worker_pipeline::10.245*1.99] mapped 1559 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 --MD -L -t 12 -a /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.map_ont.mmi /home/jupyter-isac/Data/reads/CHOK1IgG.fastq.gz
[M::main] Real time: 10.547 sec; CPU: 20.692 sec; Peak RSS: 6.881 GB


snakemake version :

In [2]:
rm $workdir/bam/*
output=$workdir/bam/$sample.minimap2.sorted.bam
snakemake $output

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	minimap2_align
	1

[Sun Oct 27 02:41:48 2019]
rule minimap2_align:
    input: /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.map_ont.mmi, /home/jupyter-isac/Data/reads/CHOK1IgG.fastq.gz
    output: /home/jupyter-isac/Data/bam/CHOK1IgG.minimap2.sorted.bam
    log: /home/jupyter-isac/Data/bam/CHOK1IgG.minimap2.align.log
    jobid: 0
    wildcards: dir=/home/jupyter-isac/Data, sample=CHOK1IgG

[Sun Oct 27 02:42:19 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-27T024147.556006.snakemake.log


In [7]:
ls -lh $workdir/bam

total 18M
-rw-r--r-- 1 jupyter-isac jupyter-isac  642 Oct 27 02:42 CHOK1IgG.minimap2.align.log
-rw-r--r-- 1 jupyter-isac jupyter-isac  17M Oct 27 02:42 CHOK1IgG.minimap2.sorted.bam
-rw-r--r-- 1 jupyter-isac jupyter-isac 680K Oct 27 02:42 CHOK1IgG.minimap2.sorted.bam.bai


## Variant detection
Sniffles is used to call structural variations.  
#### first round of calls

In [4]:
svdir=$workdir/sv
[ -e $svdir ]||mkdir $svdir
bam=$workdir/bam/$sample.minimap2.sorted.bam
vcftmp=$svdir/$sample.minimap2.sniffles.unsorted.vcf
vcf=$svdir/$sample.minimap2.sniffles.sorted.vcf
sniffles -m $bam -v $vcftmp -t $threads \
    --tmp_file $vcf.tmp \
    -s 2 
bcftools sort -o $vcf -T $svdir $vcftmp
rm $vcftmp

Estimating parameter...
	Max dist between aln events: 5
	Max diff in window: 56
	Min score ratio: 2
	Avg DEL ratio: 0.0755474
	Avg INS ratio: 0.0434565
Start parsing... RAZU01000074.1
	Switch Chr RAZU01000200.1
	Switch Chr RAZU01000234.1
	Switch Chr RAZU01000162.1
	Switch Chr RAZU01000050.1
	Switch Chr RAZU01000096.1
	Switch Chr RAZU01000102.1
	Switch Chr RAZU01001831.1
	Switch Chr RAZU01000073.1
	Switch Chr RAZU01000007.1
	Switch Chr RAZU01000230.1
	Switch Chr RAZU01000060.1
	Switch Chr RAZU01000012.1
	Switch Chr RAZU01000152.1
	Switch Chr RAZU01000120.1
	Switch Chr RAZU01000121.1
	Switch Chr RAZU01000159.1
	Switch Chr RAZU01000085.1
	Switch Chr RAZU01001824.1
	Switch Chr RAZU01000241.1
	Switch Chr RAZU01000133.1
	Switch Chr RAZU01000039.1
	Switch Chr RAZU01000087.1
	Switch Chr RAZU01000135.1
	Switch Chr RAZU01000137.1
	Switch Chr RAZU01000089.1
	Switch Chr RAZU01000044.1
	Switch Chr RAZU01000090.1
	Switch Chr RAZU01000045.1
	Switch Chr RAZU01000140.1
	Switch Chr RAZU01000047.1
	Switc

In [8]:
vcf=$workdir/sv/$sample.minimap2.sniffles.sorted.vcf
rm $vcf
snakemake -j 12 -p $vcf

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	sniffles_sv
	1	sort_vcf
	2

[Sun Oct 27 17:30:24 2019]
rule sniffles_sv:
    input: /home/jupyter-isac/Data_tmp/bam/CHOK1IgG.minimap2.sorted.bam
    output: /home/jupyter-isac/Data_tmp/sv/CHOK1IgG.minimap2.sniffles.unsorted.vcf
    log: /home/jupyter-isac/Data_tmp/sv/CHOK1IgG.minimap2.sniffles.log
    jobid: 1
    wildcards: dir=/home/jupyter-isac/Data_tmp, sample=CHOK1IgG.minimap2
    threads: 12

sniffles -m /home/jupyter-isac/Data_tmp/bam/CHOK1IgG.minimap2.sorted.bam -v /home/jupyter-isac/Data_tmp/sv/CHOK1IgG.minimap2.sniffles.unsorted.vcf -t 12 --tmp_file /home/jupyter-isac/Data_tmp/sv/CHOK1IgG.minimap2.sniffles.tmp -s 2  &> /home/jupyter-isac/Data_tmp/sv/CHOK1IgG.minimap2.sniffles.log
[Sun Oct 27 17:30:26 2019]
Finished job 1.
1 of 2 steps (50%) done

[Sun Oct 27 17:30:26 2019]
rule sort_vcf:
    input: /home/jupyter-isac/Data_tmp/sv/CHOK1Ig

In [9]:
ls -lh $vcf

-rw-r--r-- 1 jupyter-isac jupyter-isac 114K Oct 27 17:30 /home/jupyter-isac/Data_tmp/sv/CHOK1IgG.minimap2.sniffles.sorted.vcf


#### merge calls

In [26]:
vcfs=$(find $workdir/sv -name "*sniffles.sorted.vcf")
raw=$workdir/sv/raw_calls.txt
merged=$workdir/sv/SURVIVOR_merged_1kbpdist_typesave.vcf
echo $vcfs | tr " " "\n" > $raw
SURVIVOR merge $raw 1000 1 1 -1 -1 -1 $merged

merging entries: 103
merging entries: 27


In [28]:
merged=sv/SURVIVOR_merged_1kbpdist_typesave.vcf
snakemake -j 12 -p $merged

Building DAG of jobs...
Nothing to be done.
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-27T181442.997799.snakemake.log


#### force calls for SVs in merged calls

In [30]:
bam=$workdir/bam/$sample.minimap2.sorted.bam
merged=$workdir/sv/SURVIVOR_merged_1kbpdist_typesave.vcf
vcftmp=$workdir/sv/$sample.SURVIVOR.unsorted.vcf
vcf=$workdir/sv/$sample.SURVIVOR.sorted.vcf
sniffles -m $bam -v $vcf -t $threads \
    --Ivcf $merged --tmp_file \
    $workdir/sv/$samp.SURVIVORsniffles.tmp \
    -s 2 -n -1 --genotype --cluster

Automatically enabling genotype mode
Force calling SVs
Estimating parameter...
	Max dist between aln events: 5
	Max diff in window: 50
	Min score ratio: 2
	Avg DEL ratio: 0.080656
	Avg INS ratio: 0.0416317
Construct Tree...
		93 SVs found in input.
	Invalid types found skipping 0 entries.
Start parsing: Chr RAZU01000074.1
	Switch Chr RAZU01000102.1
	Switch Chr RAZU01000002.1
	Switch Chr 4_0cdhfr_vrc01wtg1m3_dgv
Finalizing  ..
Start phasing: 
	Cleaning tmp files
Start genotype calling:
	Construct tree
	Update reference alleles
		Scanning CHR RAZU01000102.1
		Scanning CHR RAZU01000002.1
		Scanning CHR 4_0cdhfr_vrc01wtg1m3_dgv
	Writing SV calls
	Cleaning tmp files


In [ ]:
raw=sv/SURVIVORsniffles_calls.txt
vcf=sv/merged_final_SURVIVOR_1kbpdist_typesave.vcf
echo {input} | tr \" \" \"\\n\" > {output.raw}
SURVIVOR merge {output.raw} 1000 -1 1 -1 -1 -1 {output.vcf}

In [12]:
vcf_final=sv/merged_final_SURVIVOR_1kbpdist_typesave.vcf
snakemake -j 12 $vcf_final

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	merge_sv_before_forcecall
	1	merge_sv_final
	2	minimap2_align
	2	sniffles_sv
	2	sniffles_sv_forcecall
	4	sort_vcf
	12

[Mon Oct 28 01:18:52 2019]
rule minimap2_align:
    input: /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.map_ont.mmi, reads/CHOK1IgG.fastq.gz
    output: bam/CHOK1IgG.minimap2.sorted.bam
    log: bam/CHOK1IgG.minimap2.align.log
    jobid: 5
    wildcards: sample=CHOK1IgG
    threads: 12

[Mon Oct 28 01:19:07 2019]
Finished job 5.
1 of 12 steps (8%) done

[Mon Oct 28 01:19:07 2019]
rule minimap2_align:
    input: /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.map_ont.mmi, reads/CHOK1host.fastq.gz
    output: bam/CHOK1host.minimap2.sorted.bam
    log: bam/CHOK1host.minimap2.align.log
    jobid: 7
    wildcards: sample=CHOK1host
    threads: 12

[Mon Oct 28 01:19:17 2019]
Finished job 7.
2 of 12 

In [13]:
grep "=TRA" $workdir/$vcf_final | cut -f 10,11

1/1:NA:6208924:4,19:++:.:TRA:9_0:NA:NA:RAZU01000002.1_6220834-4_0cdhfr_vrc01wtg1m3_dgv_11910	0/0:NA:6208924:2,0:++:.:TRA:9:NA:NA:RAZU01000002.1_6220834-4_0cdhfr_vrc01wtg1m3_dgv_11910
0/1:NA:7778025:2,4:--:.:TRA:26_0:NA:NA:RAZU01000002.1_7778043-4_0cdhfr_vrc01wtg1m3_dgv_18	./.:NA:7778025:0,0:--:.:TRA:26:NA:NA:RAZU01000002.1_7778043-4_0cdhfr_vrc01wtg1m3_dgv_18
0/1:NA:7784350:7,21:++:.:TRA:33_5:NA:NA:RAZU01000002.1_7795913-4_0cdhfr_vrc01wtg1m3_dgv_11563	0/0:NA:7784350:1,0:++:.:TRA:50:NA:NA:RAZU01000002.1_7795913-4_0cdhfr_vrc01wtg1m3_dgv_11563
0/1:NA:20003502:2,4:+-:.:TRA:0_1:NA:NA:RAZU01000102.1_20003514-4_0cdhfr_vrc01wtg1m3_dgv_12	0/0:NA:20003502:1,0:+-:.:TRA:1:NA:NA:RAZU01000102.1_20003514-4_0cdhfr_vrc01wtg1m3_dgv_12
0/1:NA:19993742:1,2:-+:.:TRA:2:NA:NA:RAZU01000102.1_20005831-4_0cdhfr_vrc01wtg1m3_dgv_12089	0/0:NA:19993742:1,0:-+:.:TRA:2:NA:NA:RAZU01000102.1_20005831-4_0cdhfr_vrc01wtg1m3_dgv_12089


## Metylation calling
We use nanopolish for methylation calling 
### nanopolish index
First step is to index the reads

In [55]:
rdir=$workdir/reads/CHOK1IgG
fq=$workdir/reads/CHOK1IgG.fastq.gz
nanopolish index -d $rdir $fq

rdir=$workdir/reads/CHOK1host
fq=$workdir/reads/CHOK1host.fastq.gz
nanopolish index -d $rdir $fq

[readdb] indexing /home/jupyter-isac/Data/reads/CHOK1IgG
[readdb] num reads: 1567, num reads with path to fast5: 1567


In [14]:
output=reads/CHOK1IgG.fastq.gz.index.readdb
snakemake $output
output=reads/CHOK1host.fastq.gz.index.readdb
snakemake $output

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	nanopolish_index
	1

[Mon Oct 28 01:42:21 2019]
rule nanopolish_index:
    input: reads/CHOK1IgG.fastq.gz
    output: reads/CHOK1IgG.fastq.gz.index.readdb
    jobid: 0
    wildcards: pre=reads/CHOK1IgG

[Mon Oct 28 01:42:22 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-28T014221.120652.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	nanopolish_index
	1

[Mon Oct 28 01:42:23 2019]
rule nanopolish_index:
    input: reads/CHOK1host.fastq.gz
    output: reads/CHOK1host.fastq.gz.index.readdb
    jobid: 0
    wildcards: pre=reads/CHOK1host

[Mon Oct 28 01:42:23 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/lo

In [15]:
ls -lh $workdir/reads

total 42M
drwxr-xr-x 2 jupyter-isac jupyter-isac 480K Oct 28 01:18 CHOK1IgG
-rw-r--r-- 1 jupyter-isac jupyter-isac  25M Oct 28 01:18 CHOK1IgG.fastq.gz
-rw-r--r-- 1 jupyter-isac jupyter-isac 7.3M Oct 28 01:42 CHOK1IgG.fastq.gz.index
-rw-r--r-- 1 jupyter-isac jupyter-isac 193K Oct 28 01:42 CHOK1IgG.fastq.gz.index.fai
-rw-r--r-- 1 jupyter-isac jupyter-isac 6.4K Oct 28 01:42 CHOK1IgG.fastq.gz.index.gzi
-rw-r--r-- 1 jupyter-isac jupyter-isac 471K Oct 28 01:42 CHOK1IgG.fastq.gz.index.readdb
-rw-r--r-- 1 jupyter-isac jupyter-isac   94 Oct 28 01:42 CHOK1IgG.index.log
-rw-r--r-- 1 jupyter-isac jupyter-isac 582K Oct 28 01:18 CHOK1IgG.readdb
drwxr-xr-x 2 jupyter-isac jupyter-isac 124K Oct 28 01:18 CHOK1host
-rw-r--r-- 1 jupyter-isac jupyter-isac 5.9M Oct 28 01:18 CHOK1host.fastq.gz
-rw-r--r-- 1 jupyter-isac jupyter-isac 1.8M Oct 28 01:42 CHOK1host.fastq.gz.index
-rw-r--r-- 1 jupyter-isac jupyter-isac  52K Oct 28 01:42 CHOK1host.fastq.gz.index.fai
-rw-r--r-- 1 jupyter-isac jupyter-isac 1.6K Oct 28

### call methylation

In [58]:
outdir=$workdir/methylation
[ -e $outdir ]||mkdir $outdir

sample=CHOK1IgG
fastq=$workdir/reads/$sample.fastq.gz
bam=$workdir/bam/$sample.minimap2.sorted.bam
output=$outdir/$sample.cpg.meth.tsv.gz

nanopolish call-methylation -v -t ${threads} -q cpg \
    -g $reference -r $fastq -b $bam |\
    gzip > $output

[post-run summary] total reads: 6388, unparseable: 0, qc fail: 1, could not calibrate: 3, no alignment: 867, bad fast5: 0


In [20]:
rm $workdir/methylation/*
sample=CHOK1IgG
output=methylation/$sample.cpg.meth.tsv.gz
snakemake -j $threads -p $output

sample=CHOK1host
output=methylation/$sample.cpg.meth.tsv.gz
snakemake -j $threads -p $output

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	call_cpg
	1

[Mon Oct 28 02:02:36 2019]
rule call_cpg:
    input: reads/CHOK1IgG.fastq.gz, bam/CHOK1IgG.minimap2.sorted.bam, reads/CHOK1IgG.fastq.gz.index.readdb
    output: methylation/CHOK1IgG.cpg.meth.tsv.gz
    jobid: 0
    wildcards: sample=CHOK1IgG
    threads: 12

nanopolish call-methylation -v -t 12 -q cpg -g /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.fa -r reads/CHOK1IgG.fastq.gz -b bam/CHOK1IgG.minimap2.sorted.bam | gzip > methylation/CHOK1IgG.cpg.meth.tsv.gz
[post-run summary] total reads: 8112, unparseable: 0, qc fail: 1, could not calibrate: 4, no alignment: 977, bad fast5: 0
[Mon Oct 28 02:04:14 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-28T020236.086467.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12

In [24]:
ls -lh $workdir/methylation

total 4.9M
-rw-r--r-- 1 jupyter-isac jupyter-isac 3.8M Oct 28 02:04 CHOK1IgG.cpg.meth.tsv.gz
-rw-r--r-- 1 jupyter-isac jupyter-isac 1.1M Oct 28 02:04 CHOK1host.cpg.meth.tsv.gz


#### make methylation bed files

In [32]:
sample=CHOK1IgG
mbed=methylation/$sample.cpg.meth.bed.gz
snakemake $mbed
sample=CHOK1host
mbed=methylation/$sample.cpg.meth.bed.gz
snakemake $mbed

Building DAG of jobs...
Nothing to be done.
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-28T021700.217936.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	mtsv_to_mbed
	1

[Mon Oct 28 02:17:01 2019]
rule mtsv_to_mbed:
    input: methylation/CHOK1host.cpg.meth.tsv.gz, /mnt/ref/Cricetulus_griseus_picr.CriGri-PICR.dna.toplevel.nihIgG.fa
    output: methylation/CHOK1host.cpg.meth.bed.gz
    jobid: 0
    wildcards: sample=CHOK1host

[Mon Oct 28 02:17:13 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-28T021700.817836.snakemake.log


#### make methylation frequency files

In [36]:
sample=CHOK1IgG
mfreq=methylation/$sample.cpg.mfreq.txt.gz
snakemake $mfreq

sample=CHOK1host
mfreq=methylation/$sample.cpg.mfreq.txt.gz
snakemake $mfreq

Building DAG of jobs...
Nothing to be done.
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-28T022732.044701.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	mbed_to_mfreq
	1

[Mon Oct 28 02:27:32 2019]
rule mbed_to_mfreq:
    input: methylation/CHOK1host.cpg.meth.bed.gz
    output: methylation/CHOK1host.cpg.mfreq.txt.gz
    log: methylation/CHOK1host.cpg.mfreq.log
    jobid: 0
    wildcards: sample=CHOK1host, mod=cpg

[Mon Oct 28 02:27:33 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-28T022732.628517.snakemake.log


#### make methylation bam

In [41]:
sample=CHOK1host
outbam=methylation/$sample.cpg.meth.bam
snakemake -j $threads $outbam

sample=CHOK1IgG
outbam=methylation/$sample.cpg.meth.bam
snakemake -j $threads $outbam

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	bam_methylation
	1

[Mon Oct 28 03:12:04 2019]
rule bam_methylation:
    input: methylation/CHOK1host.cpg.meth.bed.gz, bam/CHOK1host.minimap2.sorted.bam
    output: methylation/CHOK1host.cpg.meth.bam
    jobid: 0
    wildcards: sample=CHOK1host
    threads: 12

[Mon Oct 28 03:12:07 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jupyter-isac/ambic-epigenome-dev/.snakemake/log/2019-10-28T031204.137443.snakemake.log
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 12
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	bam_methylation
	1

[Mon Oct 28 03:12:08 2019]
rule bam_methylation:
    input: methylation/CHOK1IgG.cpg.meth.bed.gz, bam/CHOK1IgG.minimap2.sorted.bam
    output: methylation/CHOK1IgG.cpg.meth.bam
    jobid: 0
    wildcards: sample=CHOK1IgG
    threads: 12

[Mon Oct 28 03:12:26 

In [42]:
ls $workdir/methylation

CHOK1IgG.cpg.meth.bam          CHOK1host.cpg.meth.bam
CHOK1IgG.cpg.meth.bam.bai      CHOK1host.cpg.meth.bam.bai
CHOK1IgG.cpg.meth.bed.gz       CHOK1host.cpg.meth.bed.gz
CHOK1IgG.cpg.meth.bed.gz.tbi   CHOK1host.cpg.meth.bed.gz.tbi
CHOK1IgG.cpg.meth.tsv.gz       CHOK1host.cpg.meth.tsv.gz
CHOK1IgG.cpg.mfreq.log         CHOK1host.cpg.mfreq.log
CHOK1IgG.cpg.mfreq.txt.gz      CHOK1host.cpg.mfreq.txt.gz
CHOK1IgG.cpg.mfreq.txt.gz.tbi  CHOK1host.cpg.mfreq.txt.gz.tbi
